# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import altair as alt

from gcnSurrogate.models.feastnetSurrogateModel import FeaStNet
from gcnSurrogate.readers.loadConmechGraphs import loadConmechGraphs
from gcnSurrogate.visualization.altTrussViz import plotTruss, interactiveErrorPlot
from gcnSurrogate.util.gcnSurrogateUtil import *

## 1. Load simulation data

In [4]:
# dataDir = 'data/2D_Truss_v1.3/conmech/design_7_N_1000/'
# allGraphsUnfiltered = loadConmechGraphs(dataDir)

# dataDir = 'data/tower1.0/conmech/design_5_N_1000/'
# allGraphsUnfiltered = loadConmechGraphs(dataDir, loadDims=[0])

dataDir = 'data/djmmBridge1.0/conmech/bridge_N_1000/'
allGraphsUnfiltered = loadConmechGraphs(dataDir)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.059758
std,0.012355
min,0.045266
25%,0.051543
50%,0.055906
75%,0.063586
max,0.124130


In [9]:
# tower
# plotTruss(allGraphsUnfiltered[0], 
#           showDeformed=True, 
#           defScale=100, 
#           width=80, 
#           lineWidth=1, 
#           showLoads=True, 
#           showSupports=True, 
#           loadTheta=0, 
#           loadSize=100, 
#           supportSize=100)

# bridge
plotTruss(allGraphsUnfiltered[0], 
          showDeformed=True, 
          defScale=300, 
          width=500, 
          lineWidth=0.7, 
          showLoads=True, 
          showSupports=True, 
          loadSize=100, 
          supportSize=150)

alt.LayerChart(...)

## 2. Filter and partition

In [10]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.056490
std,0.006821
min,0.045266
25%,0.051163
50%,0.055148
75%,0.060596
max,0.075970


## 3. Train a GCN

In [11]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                         epochs=100,
                         saveDir='results/gcnBridge01/')

plotHistory(history)

RuntimeError: CUDA out of memory. Tried to allocate 3.62 GiB (GPU 0; 11.17 GiB total capacity; 8.37 GiB already allocated; 2.15 GiB free; 8.73 GiB reserved in total by PyTorch)

## 4. Test the GCN

In [9]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000004,0.001400,0.074304,0.942238,0.972997,0.847795,0.0
test,0.000007,0.001609,0.083742,0.841736,0.936382,0.804915,0.0


## 5. Visualize some predictions

In [11]:
i = 4
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=20, prediction=pred)

alt.LayerChart(...)

In [10]:
# # interactive scatter plot
# alt.data_transformers.enable('json')
# allPreds = gcn.predict(testData)
# display(interactiveErrorPlot(testData, allPreds))
# alt.data_transformers.enable('default');